<a href="https://colab.research.google.com/github/karankulshrestha/ai-notebooks/blob/main/rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
data = open('sample.txt', 'r').read()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)

In [ ]:
print(f'data has total {data_size} characters and {vocab_size} unique chars')

data has total 6661 characters and 66 unique chars


In [ ]:
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}

![RNN training diagram](https://d2l.ai/_images/rnn-train.svg)

# RNN Architecture Example

**Configuration:** `vocab_size=5`, `hidden_size=10`

## Forward Pass Flow:
```
INPUT: One-hot [0,1,0,0,0]  (5 dimensions)
           ↓
       [Wxh: 5×10 = 50 weights]
           ↓
       Combine with memory
           ↓
HIDDEN: tanh(Wxh·x + Whh·h_prev + bh)  (10 neurons)
           ↓
       [Why: 10×5 = 50 weights]
           ↓
OUTPUT: softmax(Why·h + by)  (5 probabilities)
```

## Weight Matrices:
- **Wxh** (5×10): Transforms input to hidden space → 50 parameters
- **Whh** (10×10): Recurrent connections for memory → 100 parameters
- **Why** (10×5): Transforms hidden to output space → 50 parameters
- **Total weights:** 200 parameters (plus biases)

# Batch Processing with Memory Continuity

The RNN maintains memory across batches by passing the final hidden state forward.

## Batch 1: Process characters 0-24
```
─────────────────────────────────────────────────
Input:  hprev = [0, 0, 0, ...]  (blank memory)
        
        char[0] → h[0]
        char[1] → h[1]
        char[2] → h[2]
        ...
        char[24] → h[24]
        
Output: hprev = h[24]  (memory of chars 0-24)
```

## Batch 2: Process characters 25-49
```
─────────────────────────────────────────────────
Input:  hprev = h[24]  (memory from batch 1)
        
        char[25] → h[0]  (using h[24] as hs[-1])
        char[26] → h[1]
        char[27] → h[2]
        ...
        char[49] → h[24]
        
Output: hprev = h[24]  (memory of chars 0-49 now!)
```

## Batch 3: Process characters 50-74
```
─────────────────────────────────────────────────
Input:  hprev = h[24] from batch 2
        
        char[50] → h[0]
        char[51] → h[1]
        ...
        
And so on...
```

**Key Insight:** Each batch "remembers" everything from previous batches through the `hprev` state that gets passed forward!

# The COMBINATION happens INSIDE the state update:

The hidden state update formula:
```
h[t] = tanh(Wxh·x[t] + Whh·hprev + bh)
            └──┬──┘     └───┬───┘
            current    previous (hprev)
            input      memory
```

**Key insight:** We multiply `Whh · hprev` to transform and select which parts of the previous memory are relevant for the current timestep.

**Components:**
- `Wxh·x[t]` - processes the current input
- `Whh·hprev` - processes the previous hidden state (memory)
- `bh` - bias term
- `tanh` - activation function that squashes the result to [-1, 1]

In [ ]:
hidden_size = 100
seq_length = 10
learning_rate = 1e-2

In [ ]:
Wxh = np.random.randn(hidden_size, vocab_size) * 0.1
Whh = np.random.randn(hidden_size, hidden_size) * 0.1
Why = np.random.randn(vocab_size, hidden_size) * 0.1

bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))

# Gradient Flow in Backpropagation Through Time
```
dh = np.dot(Why.T, dy) + dhnext
     └──────┬──────┘     └──┬──┘
     │                   │
     │                   └─ Error from h[1]→h[2]→h[3]→h[4]
     │                      (future timesteps)
     │
     └─ Error from h[1]→y[1]
        (current prediction)
```

In [ ]:
def loss_fun(inputs, targets, hprev):
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)

    loss = 0
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1))
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh)
        ys[t] = np.dot(Why, hs[t]) + by
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))
        loss += -np.log(ps[t][targets[t], 0])

    # calculations of backward pass

    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0]) # override it not accumulate, pass the gradient of loss w.r.t hidden state from last input to first during backpropagation

    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # dy = gradient of loss w.r.t. output y[t]
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # How much does the hidden state h[t] contribute to the overall loss? h[t] → y[t] → loss[t]
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)

    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

# Random Sampling with Probabilities
```
np.random.choice([0, 1, 2, 3, 4], p=[0.05, 0.10, 0.60, 0.20, 0.05])
                 └──────┬──────┘     └──────────┬──────────┘
                 pick from these     with these probabilities
```

## How it works:
- **Indices:** `[0, 1, 2, 3, 4]` - possible character indices
- **Probabilities:** `[0.05, 0.10, 0.60, 0.20, 0.05]` - likelihood of each
- Index `2` has 60% chance (most likely to be picked)
- Indices `0` and `4` have only 5% chance each

In [ ]:
def sample(h, seed_ix, n): # h-> prev memory state
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

# Adagrad Optimizer Update Rule
```
param += -learning_rate * dparam / np.sqrt(mem + 1e-8)
         └───────┬──────┘  └──┬──┘   └──────┬─────┘
         standard SGD     gradient  adaptive scaling
```

## Adaptive Learning Rate Logic:

- **If `mem` is large** → `sqrt(mem)` is large → divide by large number → **small update**
- **If `mem` is small** → `sqrt(mem)` is small → divide by small number → **large update**

## Components:
- `learning_rate` - base step size
- `dparam` - gradient (direction to move)
- `mem` - accumulated squared gradients (memory of past updates)
- `1e-8` - small constant to prevent division by zero
- `np.sqrt(mem + 1e-8)` - adaptive scaling factor

**Key Insight:** Parameters that have received large gradients in the past get smaller updates (more conservative), while parameters with small historical gradients get larger updates (more aggressive).

In [ ]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by)  # memory variables for Adagrad

smooth_loss = -np.log(1.0 / vocab_size) * seq_length  # loss at iteration 0 -- initial random loss

while True:

    # Reset RNN memory and data pointer at end of data
    if p + seq_length + 1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size, 1))  # reset prev RNN Memory
        p = 0  # offset to the data

    # Prepare inputs and targets
    inputs = [char_to_idx[ch] for ch in data[p:p + seq_length]]  # input characters
    targets = [char_to_idx[ch] for ch in data[p + 1:p + seq_length + 1]]  # next characters

    # Print sample every 100 iterations
    if n % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(idx_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt,))

    # Forward seq_length and calculate gradients + loss
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = loss_fun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001

    # Print training progress
    if n % 100 == 0:
        print('iter %d, loss: %f' % (n, smooth_loss))

    # Perform parameter update with Adagrad
    for param, dparam, mem in zip(
        [Wxh, Whh, Why, bh, by],
        [dWxh, dWhh, dWhy, dbh, dby],
        [mWxh, mWhh, mWhy, mbh, mby]
    ):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8)  # Adagrad update

    p += seq_length  # move data pointer
    n += 1  # iteration counter

----
 ?AY)4.; yB2-HLIbPk
rf:i;Y(w3gsxUNS(a.1:UGmd;m;3meghxu6yAI9sYfy,0Ot);65,YbFqNaq2TsRtaxbUjOR;coB18je:MI?WM3k2t7uclj5BBOyTzjrh1gDvyh.a0j5azg9)sRFYRb:TUCBLHGc8mUBPN:T(Mo;UYRR'gD5LaML.iedWUn8(of25gy83gcvM8 
----
iter 0, loss: 41.897431
----
 ' blasarei e Wonpa dpd Tslee w'e T o,a:teet enegr,  iyTl-heketat ioi6ehehWaueteu1u o
e'unT rareh,a H-heeesdoi' re l(ns v
ossWSt  T Wwle,u   dae 
eeTr 1ngeh
 eedstsat ehe u
lyeioeypbealsnnhivfbCn,aoo
r 
----
iter 100, loss: 40.837761
----
 m rpmm be ha t  a e s t r ewom  sfhbme : g  det
nc uoes ai aiit,  w  Fd  el hwssy  is iheehpg
    ul
.edeee  i:  iw eFbnn
  
 t lh  c yaeg   ose   edi w  cyteei
e  eht ,s  s ehio ei n hg  tniocse  si  
----
iter 200, loss: 39.689422
----
 esctew,gh(oLntee e  ir  hhb lga7th hi T,told lhvnyt;nu
 ,radfot oxf res thpHey  Ftswetsfoht t o2t emm  irei uae yswsase,otvuch  ew Tl Af is
ho heen nIlau
eisnusu uunte gB5
eamaht yhuyhOe m mbpNe sove  
----
iter 300, loss: 38.655627
----
 
owsf
th
  sWy iiefhe  tna anhui tveio n